In [1]:
#coding:utf-8
"""
相机校正
学会用棋盘格进行相机校正（张正友标定法）
熟悉opencv相关函数
cv2.findChessboardCorners
cv2.cornerSubPix
cv2.drawChessboardCorners
cv2.calibrateCamera
cv2.undistort
"""
import numpy as np
import cv2

def calibrate_camera():
    #每个校准图像映射到棋盘角到数量
    objPoints = {
        1: (9, 5),
        2: (9, 6),
        3: (9, 6),
        4: (9, 6),
        5: (9, 6),
        6: (9, 6),
        7: (9, 6),
        8: (9, 6),
        9: (9, 6),
        10: (9, 6),
        11: (9, 6),
        12: (9, 6),
        13: (9, 6),
        14: (9, 6),
        15: (9, 6),
        16: (9, 6),
        17: (9, 6),
        18: (9, 6),
        19: (9, 6),
        20: (9, 6),
    }
    #目标点空间坐标
    obj3DList = []
    
    #图像中棋盘格中的2D点
    obj2DList = []

    for k in objPoints:
        nx, ny = objPoints[k]
        ######棋盘格对应3D坐标点，x为0-8， y为0-4（对应棋盘格横着9个点，纵着5个点）， z = 0
        obj = np.zeros((nx * ny, 3), np.float32)

        obj[:, :2] = np.mgrid[0: nx, 0: ny].T.reshape(-1, 2)
#         print(obj)
#         print(obj.shape)
        fname = 'camera_cal_pic/calibration%s.jpg' % str(k)
        img = cv2.imread(fname)
        #将图像转换为灰度图
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        ########查找角点，利用cv2.findChessboardCorners函数，函数返回
        ####ret：是否查找到； corners：角点坐标
        #####################填空1 （一行代码）#####################################
        """
        int cvFindChessboardCorners( const void* image, CvSize pattern_size, CvPoint2D32f* corners, int* corner_count=NULL,
                                        int flags=CV_CALIB_CB_ADAPTIVE_THRESH );
        input:
            image：输入的棋盘图，必须是8位的灰度图或者彩色图像；
            pattern_size：棋盘图中每行和每列角点的个数；
            corners:检测到的角点;
            corner_count:输出，角点的个数。如果不是NULL，函数将检测到的角点的个数存储于此变量;
            Flags:各种操作标志，可以是0或者下面值的组合：
                CV_CALIB_CB_ADAPTIVE_THRESH -使用自适应阈值（通过平均图像亮度计算得到）将图像转换为黑白图，而不是一个固定的阈值;
                CV_CALIB_CB_NORMALIZE_IMAGE -在利用固定阈值或者自适应的阈值进行二值化之前，先使用cvNormalizeHist来均衡化图像亮度;
                CV_CALIB_CB_FILTER_QUADS -使用其他的准则（如轮廓面积，周长，方形形状）来去除在轮廓检测阶段检测到的错误方块;
            补充说明：
                函数cvFindChessboardCorners试图确定输入图像是否是棋盘模式，并确定角点的位置。如果所有角点都被检测到且它们都被以一定顺序排布，
            函数返回非零值，否则在函数不能发现所有角点或者记录它们地情况下，函数返回0。例如一个正常地棋盘图右8x8个方块和7x7个内角点，内角
            点是黑色方块相互联通的位置。这个函数检测到地坐标只是一个大约的值，如果要精确地确定它们的位置，可以使用函数cvFindCornerSubPix。
        output:
            ret:是否查找到角点；
            corner:角点坐标；
        """
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        
        #########################################################################

        if ret == True:
            obj3DList.append(obj)
            #利用cv2.cornerSubPix可以更精细的查找角点坐标，如果查找到了，用这个，没查找到用cv2.findChessboardCorners中找到的角点
            corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria=(cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 30, 0.001))
            if corners2.any():
                obj2DList.append(corners2)
            else:
                obj2DList.append(corners)
            #可以利用cv2.drawChessboardCorners显示每张图查找到的角点的坐标
            # cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            # cv2.imshow("img", img)
            # cv2.waitKey(0)
        else:
            print('Warning: ret = %s for %s' % (ret, fname))

    img = cv2.imread('testImage/straight_lines1.jpg')
    img_size = (img.shape[1], img.shape[0])
    #利用图像中2d点和空间3d点计算旋转和平移矩阵，函数使用cv2.calibrateCamera，返回mtx（相机内参矩阵）, dist（畸变矩阵）
    ################填空2（一行代码）################################################
    """
    
    mtx:     内参数矩阵
    dist:    畸变系数
    rvecs:   旋转向量  
    tvecs:   平移向量  
    """
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj3DList, obj2DList, img_size, None, None)
    ##############################################################################
    

    return mtx, dist

mtx, dist = calibrate_camera()
print ("mtx, dist", mtx, dist)
img = cv2.imread('./camera_cal_pic/calibration2.jpg')

##########将img进行校正，利用cv2.undistort这个函数，根据相机内参和外参进行相机校正，校正后的图像为dst

####################填空3（一行代码）#################################################
"""
直接得到校正之后的图像
"""
dst = cv2.undistort(img, mtx, dist, None, mtx)
###################################################################################

cv2.imwrite('./camera_cal_pic_undistort/calibration2_undistort.jpg', dst)


mtx, dist [[1.15996332e+03 0.00000000e+00 6.62925396e+02]
 [0.00000000e+00 1.15670902e+03 3.89514521e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[-2.53009624e-01  2.09415850e-02 -2.04829701e-04 -2.77196818e-05
  -6.78247481e-02]]


True